# **CS 6390 Final Project**
*Nicholas Martucci & Anish Moorthy*

In [0]:
%cd ~
%pwd
!rm -rf ~/6390-project
!git clone https://github.com/anlsh/6390-project.git
%cd 6390-project
!git init
!git pull
!git checkout master

!git status
!git log -1

from affine_checker import AffineTypeChecker
from interpreter import evaluate
from dsl_parser import dsl_parse
from test_interpreter import base_env as get_base_runtime_env
from test_affine_checker import base_tcheck_env

def p_eval(src_str):
  return evaluate(get_base_runtime_env(), dsl_parse(src_str))

def p_tcheck(src_str):
  return AffineTypeChecker.type_check(base_tcheck_env(), dsl_parse(src_str), 
                                      descope=True)

## **Project Objective**

The goal for this project was to implement and prove properties of a borrow checker that behaves similarly to Rust's borrow system. Both of us wanted to learn more about Rust's properties and the internals that provide these properties. Over time the project evolved into a type checker that supports both linear and affine type checking of a toy programming language.

## **Resources**

To learn more about Affine and Linear Type Systems, we used [David Walker's chapter](http://mitp-content-server.mit.edu:18180/books/content/sectbyfn?collid=books_pres_0&id=1104&fn=9780262162289_sch_0001.pdf) from Advanced Topic in Types and Programming Languages as a resource

* Overview of Substructural Type Systems
* Deals mostly with Linear Types
* Algorithmic type checking

## **Substructural Types**

Linear types forbid the structural typing rules of weakening and contraction, so every linear object must be used exactly once.

Affine types only forbid contraction, so affine objects can be used at most once.

## **DSL: The Dead-Simple Language**

Defined a toy programming language for the purposes of this project.
* Originally aimed to build a type-checker for subset of Common Lisp
* Kept Lisp-esque syntax
* Language is evaluated and type-checked by maintaining for each scope an environment that maps bindings of names to values
  * Interpreter bindings are
    * Var name -> value
    * Ref name -> binding of referenced var
  * Type-checker bindings are
    * Var name -> type
    * Ref name -> type of referenced var
* The mapping of function names to function bodies is maintained separately so that functions defined in an outer scope can be called in an inner scope 

Language has just a few constructs

In [0]:
#########################
# Sequential Evaluation #
#########################
prog = """
(3 true)
"""
p_eval(prog)

In [0]:
#######################
# Defining a variable #
#######################
prog = """
(
  (defvar x _ 3)
  x
)
"""
p_eval(prog)


In [0]:
##################################
# Defining and calling functions #
##################################
prog = """
(
  (defun sum-plus-two _ 
         (
           (x _)
           (y _)
         )
     (apply + x (apply + y 2))
  )
  (apply sum-plus-two 3 5)
)
"""
p_eval(prog)

In [0]:
#################
# If statements #
#################
prog = """
(
  (if true -30 30)
)
"""
p_eval(prog)

In [0]:
###############
# While loops #
###############
prog = """
(
  (defvar x _ 0)
  (while (apply < x 50) 
         0 
        (set x (apply + x 1))
  )
  x
)
"""
p_eval(prog)

In [0]:
##############
# References #
##############
prog = """
(
  (defvar x _ 0)
  (defun set-ref-100 _ ((numref _))
      (setrefval numref 100)
  )
  (scope 
      (defvar xref _ (mkref x))
      (apply set-ref-100 xref)
  )
  x
)
"""
p_eval(prog)

## Recursion

In [0]:
# Fibonacci: 0, 1, 1, 2, 3...

prog = """
(
    (defun fib _ ((n _))
        (defvar ret _ 0) 
        (if (apply = 0 n)
            (set ret 0)
            (if (apply = 1 n)
                (set ret 1)
                (set ret (apply +  (apply fib (apply - n 1))  (apply fib (apply - n 2))  ))
            )
        )
        ret
    )
   (apply fib 4)
)
"""
p_eval(prog)

## **Built-in Functions in DSL**


```
builtin_fn_vals = {
    # Standard arithmetic operators and comparisons
    '+': op.add, '-': op.sub, '*': op.mul, '/': op.truediv,
    '>': op.gt, '<': op.lt, '>=': op.ge, '<=': op.le, 

    # Standard logical operators
    'not': op.not_, 'or': op.or_, 'and': op.and_,

    # Testing objects for equality
    '=': op.eq,

    # Functions for working with files
    'fopen': lambda id: open(str(id), "w+"),
    'fwrite': lambda f, out: f[1][0].write(str(out) + "\n"),
    'fclose': lambda f: f.close(),
}
```



## **Type-Checking**
What are all of the underscores in the above programs? That's where all of the type specifiers go!

* Type-checking is totally seperate from runtime

A "type specifier" has three parts:
* Modifier (Linear, Affine, Unrestricted)
* Typeclass (either "value" or "reference")
* Type arguments

Must declare types for
* Functions (return values and arguments)
* Variables

In [0]:
# Evaluator will happily run this code
prog = """
(apply + 3 true)
"""
p_eval(prog)

In [0]:
# But it won't pass the type checker
prog = """
(apply + 3 true)
"""
p_tcheck(prog)

## **Unrestricted Type-Checking in DSL**

DSL's simple semantics allow an elegant type-checking/inferring algorithm

* Defvar: Ensure that inferred type of definition form is subtype of declared type
* Apply: Ensure that function is called with right number of arguments and that argument types are subtypes of those declared in signature
* Defun: Assume that arguments have the types in signature, then infer the type of the function body and assert that final type is of correct kind

In [0]:
prog = """
(
  (defun add-false-nocast (un val int)
                          ( (n1 (un val int)) (n2 (un val bool))  )

      (apply + n1 n2)
  )
)
"""
p_tcheck(prog)

In [0]:
prog = """
(
  (defun bool2int (un val int) 
                  ((b (un val bool))) 
     (if b 1 0)
  )

  (defun add-false-nocast (un val int)
                           ( (n1 (un val int)) (n2 (un val bool))  )

      (apply + n1 (apply bool2int n2))
  )
)
"""
p_tcheck(prog)

**Type-Checking Recursive Functions**

* No attempt to enforce well-formed recursion through types

So the type-annotated recursive Fibonacci type-checks...

In [0]:
# Fibonacci: 0, 1, 1, 2, 3...

prog = """
(
    (defun fib (un val int) ((n (un val int)))
        (defvar ret (un val int) 0) 
        (if (apply = 0 n)
            (set ret 0)
            (if (apply = 1 n)
                (set ret 1)
                (set ret (apply +  (apply fib (apply - n 1))  (apply fib (apply - n 2))  ))
            )
        )
        ret
    )
   (apply fib 4)
)
"""
p_tcheck(prog)

... but so does this

In [0]:
prog = """
(
    (defun fib (un val int) ((n (un val int)))
        (apply fib n)
    )
)
"""
p_tcheck(prog)

**Type-Checking If statements and While loops**

In order to simplify the process of type checking control flow branches and loops, we impose the restriction that the environment must not be modified by either branch of the if statement or the body of the while loop. 

So this if statement will type-check:

In [0]:
prog = """
(
  (defvar x (un val int) 3)
  (defvar y (un val int) 3) 
  (if true (set x (apply + x 1)) 
           (set y (apply - y 1)))
  (apply + x 0)
)
"""
p_tcheck(prog)

but this if statement does not because the file is closed in only one of the branches:

In [0]:
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (if true (apply fclose f) 
           ((defvar fref (un ref (lin val file)) (mkref f))
           (apply fwrite fref 1)))
)
"""
p_tcheck(prog)

## **Linear & Affine Type-Checking in DSL**

DSL supports fully-fledged linear and affine type-checking

Can be achieved through slight modifications to the type-checking mechanism

* When a variable marked as linear/affine is type-checked, mark as "used"
* At the end of a scope check for linear judgements at uppermost level
* Linear and affine variables are consumed when applied to a function, akin to the move semantics of Rust

Scopes stack, and are introduced by
* Function definitions (little bit special)
* Branches of the if statement
* The "scope" operator

In [0]:
prog = """
(
   (defvar x (lin val int) 3)
)
"""
p_tcheck(prog)

In [0]:
prog = """
(
  (defun consume-int (un val int) 
                      ( (n (lin val int))  )
       (apply + 0 n)
   )
   (defvar x (lin val int) 3)
   (apply consume-int x)
)
"""
p_tcheck(prog)

In [0]:
####################################################################
# Of course, a linear/affine variable can't be used more than once #
####################################################################

prog = """
(
   (defvar x (lin val int) 3)
   (apply + 0 x)
   x
)
"""
p_tcheck(prog)

In [0]:
##################################################
# Can give a used linear variable a new value... #
##################################################
prog = """
(
  (defvar x (lin val int) 3)
  (set x (apply + x 1))

  (apply + x 0)
)
"""
p_tcheck(prog)

In [0]:
#########################
# ...but only used ones #
#########################
prog = """
(
  (defvar x (lin val file) (apply fopen 300))
  (set x (apply fopen 400))
)
"""
p_tcheck(prog)

## Note: DSL doesn't allow linear values to be "orphaned"


In [0]:
# So the following code is fine...
prog = """
(
  (defvar x (un val int) 3)
   x
   3000
)
"""
p_tcheck(prog)

In [0]:
# ... but this is not.
prog = """
(
  (defvar x (lin val int) 3)
   x
   3000
)
"""
p_tcheck(prog)

Implementation: make sure that every instance of a linear variable being used is associated with a binding

Bindings are established by
* set
* defvar
* apply

# Linear types obviously good for resource management
* Files, heap-allocated memory, etc.
* Guarantee a file is closed exactly once

## Types of the DSL built-in functions
```
builtin_fn_types = {
    arithmetic : un LIN_INT -> LIN_INT -> UN_INT,
    comparisons: un LIN_INT -> LIN_INT -> UN_BOOL,

    ... etc

    "fopen": un LIN_INT -> LIN_FILE,
    "fwrite": un &LIN_FILE -> LIN_INT -> UNIT,
    "fclose": un LIN_FILE -> UNIT
}
```

## **How do you use a linear/affine variable multiple times?**

We would like to be able to write to files multiple times, but only "consume" the file once when it is closed. 

This is where references come in! They allow "borrowing" a value.

In [0]:
# Can write to files multiple times!
%rm 300
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (scope 
    (defvar fref (un ref (lin val file)) (mkref f))
    (apply fwrite fref 1)
    (apply fwrite fref 2)
  )
  (apply fclose f)
)  
"""
p_tcheck(prog)
p_eval(prog)
!echo "Printing the contents of 300.txt!"
%cat 300

In [0]:
# While a reference to an variable is active, the original is unusable
%rm 300
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (scope 
    (defvar fref (un ref (lin val file)) (mkref f))
    (apply fclose f)
  )
)  
"""
p_tcheck(prog)

## **Important attributes of DSL references**

There is a dereferencing operator, but it can only work on references to 
unrestricted objects.

In [0]:
prog = """
(
  (defvar x (un val int) 3)
  (scope 
    (defvar xref (un ref (un val int)) (mkref x))
    (apply + 3 (deref xref))
  )
)
"""
p_tcheck(prog)
p_eval(prog)

In [0]:
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (scope 
    (defvar fref (un ref (lin val file)) (mkref f))
    (apply fclose (deref fref))
  )
)
"""
p_tcheck(prog)

References are affine-ish
* Can be used multiple times
* Cannot be duplicated
* Only one reference to a variable may be live at a time

Akin to mutable references in Rust

In [0]:
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (scope 
    (defvar fref (un ref (lin val file)) (mkref f))
    (defvar fref2 (un ref (lin val file)) (mkref f))
  )
)
"""
p_tcheck(prog)

In [0]:
prog = """
(
  (defvar f (lin val file) (apply fopen 300))
  (scope 
    (defvar fref (un ref (lin val file)) (mkref f))
    (defvar fref2 (un ref (lin val file)) fref)
    (apply fwrite fref 987)
  )
  (apply fclose f)
)
"""
p_tcheck(prog)

In [0]:
prog = """
(
  (defun wants-two-frefs (un val int)
                         ( 
                           (fref1 (un ref (lin val file)))
                           (fref2 (un ref (lin val file)))
                         )
     3
  )
  (defvar f (lin val file) (apply fopen 300))
  (scope 
      (defvar fref (un ref (lin val file)) (mkref f))
      (apply wants-two-frefs fref fref)
  )
)
"""
p_tcheck(prog)

## **Future Work**

* Write up formal typing rules
* Allow declaration of the "copy" trait
* Explicit "return" statement

## **More Future Work**
* Sum and Product Types
* Automatic return of references to objects in same scope
